In [1]:
include("../QR/housQR.jl")
include("../utilities/genFunc.jl")
include("../L-BFGS/OracleFunction.jl")
include("../L-BFGS/LBFGS.jl")
include("../L-BFGS/BFGS.jl")
include("../L-BFGS/SR1.jl")
using .LBFGS
using .BFGS
using .SR1
using .OracleFunction
using .housQR
using LinearAlgebra, BenchmarkTools, CSV, DataFrames

baseDir = "results/comparison";
mkpath(baseDir);

In [ ]:
# QR vs LBFGS running time comparison with respect to n, fixing m, on ill-conditioned matrix
λ    = 1e-12
m    = 200
runs = 10

# output csv
outputvsc = joinpath(baseDir, "QRvsLBFGS-n-m" * string(m) * "-illcond--time.csv");
accData = Dict(
    :n   => Array{Float64}(undef, 0),
    :meantimeLBFGS => Array{Float64}(undef, 0),
    :stdtimeLBFGS => Array{Float64}(undef, 0),
    :meanallocsLBFGS => Array{Float64}(undef, 0),
    :meantimeQR => Array{Float64}(undef, 0),
    :stdtimeQR => Array{Float64}(undef, 0),
    :meanallocsQR => Array{Float64}(undef, 0)
    )

for n ∈ (0:runs) .* 500 .+ 500
    gf = genFunc(:exactRandDataset, λ=λ, m=m, n=n)

    t_qr = @benchmark qrfact($gf[:X_hat]) \ $gf[:y_hat] samples=10 evals=1

    ls = LeastSquaresF(gf)

    t_lbfgs = @benchmark LimitedMemoryBFGS($ls) samples=10 evals=1
    
    push!(accData[:n], n)
    push!(accData[:meantimeLBFGS], mean(t_lbfgs.times))
    push!(accData[:stdtimeLBFGS], std(t_lbfgs.times))
    push!(accData[:meanallocsLBFGS], mean(t_lbfgs.memory))
    push!(accData[:meantimeQR], mean(t_qr.times))
    push!(accData[:stdtimeQR], std(t_qr.times))
    push!(accData[:meanallocsQR], mean(t_qr.memory))
    println("Done: n " * string(n))
    flush(stdout)
end

# CSV.write(outputvsc, DataFrame(accData));

In [ ]:
# QR vs LBFGS running time comparison with respect to n, fixing m, on well-conditioned matrix
λ    = 1e-4
m    = 200
runs = 10

# output csv
outputvsc = joinpath(baseDir, "QRvsLBFGS-n-m" * string(m) * "-wellcond--time.csv");
accData = Dict(
    :n   => Array{Float64}(undef, 0),
    :meantimeLBFGS => Array{Float64}(undef, 0),
    :stdtimeLBFGS => Array{Float64}(undef, 0),
    :meanallocsLBFGS => Array{Float64}(undef, 0),
    :meantimeQR => Array{Float64}(undef, 0),
    :stdtimeQR => Array{Float64}(undef, 0),
    :meanallocsQR => Array{Float64}(undef, 0)
    )

for n ∈ (0:runs) .* 500 .+ 500
    gf = genFunc(:exactRandDataset, λ=λ, m=m, n=n)

    t_qr = @benchmark qrfact($gf[:X_hat]) \ $gf[:y_hat] samples=10 evals=1

    ls = LeastSquaresF(gf)

    t_lbfgs = @benchmark LimitedMemoryBFGS($ls) samples=10 evals=1
    
    push!(accData[:n], n)
    push!(accData[:meantimeLBFGS], mean(t_lbfgs.times))
    push!(accData[:stdtimeLBFGS], std(t_lbfgs.times))
    push!(accData[:meanallocsLBFGS], mean(t_lbfgs.memory))
    push!(accData[:meantimeQR], mean(t_qr.times))
    push!(accData[:stdtimeQR], std(t_qr.times))
    push!(accData[:meanallocsQR], mean(t_qr.memory))
    println("Done: n " * string(n))
    flush(stdout)
end

# CSV.write(outputvsc, DataFrame(accData));

In [5]:
# BFGS vs LBFGS running time comparison with respect to n, fixing m, on well-conditioned matrix
λ    = 1e-4
m    = 200
runs = 10

# output csv
outputvsc = joinpath(baseDir, "BFGSvsLBFGS-n-m" * string(m) * "--time.csv");
accData = Dict(
    :n   => Array{Float64}(undef, 0),
    :meantimeLBFGS => Array{Float64}(undef, 0),
    :stdtimeLBFGS => Array{Float64}(undef, 0),
    :meanallocsLBFGS => Array{Float64}(undef, 0),
    :meantimeBFGS => Array{Float64}(undef, 0),
    :stdtimeBFGS => Array{Float64}(undef, 0),
    :meanallocsBFGS => Array{Float64}(undef, 0)
    )

for i ∈ 0:runs
    n = 500 + i * 500


    t_lbfgs = @benchmark LimitedMemoryBFGS(ls) setup=(ls = (genFunc(:randDataset, λ=$λ, m=$m, n=$n) |> LeastSquaresF)) samples=10 evals=1
    t_bfgs = @benchmark BroydenFletcherGoldfarbShanno(ls) setup=(ls = (genFunc(:randDataset, λ=$λ, m=$m, n=$n) |> LeastSquaresF)) samples=10 evals=1

    push!(accData[:n], n)
    push!(accData[:meantimeLBFGS], mean(t_lbfgs.times))
    push!(accData[:stdtimeLBFGS], std(t_lbfgs.times))
    push!(accData[:meanallocsLBFGS], mean(t_lbfgs.memory))
    push!(accData[:meantimeBFGS], mean(t_bfgs.times))
    push!(accData[:stdtimeBFGS], std(t_bfgs.times))
    push!(accData[:meanallocsBFGS], mean(t_bfgs.memory))
    println("Done: n " * string(n))
    flush(stdout)
end

CSV.write(outputvsc, DataFrame(accData));

Done: n 500
Done: n 1000
Done: n 1500
Done: n 2000
Done: n 2500
Done: n 3000
Done: n 3500
Done: n 4000
Done: n 4500
Done: n 5000
Done: n 5500


In [4]:
# BFGS vs LBFGS running time comparison with respect to m, fixing n, on well-conditioned matrix
λ    = 1e-4
n    = 50
runs = 10

# output csv
outputvsc = joinpath(baseDir, "BFGSvsLBFGS-m-n" * string(n) * "--time.csv");
accData = Dict(
    :m   => Array{Float64}(undef, 0),
    :meantimeLBFGS => Array{Float64}(undef, 0),
    :stdtimeLBFGS => Array{Float64}(undef, 0),
    :meanallocsLBFGS => Array{Float64}(undef, 0),
    :meantimeBFGS => Array{Float64}(undef, 0),
    :stdtimeBFGS => Array{Float64}(undef, 0),
    :meanallocsBFGS => Array{Float64}(undef, 0)
    )

for i ∈ 0:runs
    m = 500 + i * 500


    t_lbfgs = @benchmark LimitedMemoryBFGS(ls) setup=(ls = (genFunc(:randDataset, λ=$λ, m=$m, n=$n) |> LeastSquaresF)) samples=10 evals=1
    t_bfgs = @benchmark BroydenFletcherGoldfarbShanno(ls) setup=(ls = (genFunc(:randDataset, λ=$λ, m=$m, n=$n) |> LeastSquaresF)) samples=10 evals=1

    push!(accData[:m], m)
    push!(accData[:meantimeLBFGS], mean(t_lbfgs.times))
    push!(accData[:stdtimeLBFGS], std(t_lbfgs.times))
    push!(accData[:meanallocsLBFGS], mean(t_lbfgs.memory))
    push!(accData[:meantimeBFGS], mean(t_bfgs.times))
    push!(accData[:stdtimeBFGS], std(t_bfgs.times))
    push!(accData[:meanallocsBFGS], mean(t_bfgs.memory))
    println("Done: m " * string(m))
    flush(stdout)
end

CSV.write(outputvsc, DataFrame(accData));

Done: m 500
Done: m 1000
Done: m 1500
Done: m 2000
Done: m 2500
Done: m 3000
Done: m 3500
Done: m 4000
Done: m 4500
Done: m 5000
Done: m 5500


In [ ]:
# QR vs LBFGS running time comparison with respect to m, fixing n, on well-conditioned matrix
λ    = 1e-4
n    = 50
runs = 10

# output csv
outputvsc = joinpath(baseDir, "QRvsLBFGS-m-n" * string(n) * "-wellcond--time.csv");
accData = Dict(
    :m   => Array{Float64}(undef, 0),
    :meantimeLBFGS => Array{Float64}(undef, 0),
    :stdtimeLBFGS => Array{Float64}(undef, 0),
    :meanallocsLBFGS => Array{Float64}(undef, 0),
    :meantimeQR => Array{Float64}(undef, 0),
    :stdtimeQR => Array{Float64}(undef, 0),
    :meanallocsQR => Array{Float64}(undef, 0)
    )

# warm up
qrfact([0. 1; 1 0]) \ [1.; 1]

for i ∈ 0:runs
    m = 500 + i * 500
    gf = genFunc(:exactRandDataset, λ=λ, m=m, n=n)

    t_qr = @benchmark begin
        QR = qrfact($gf[:X_hat])
        w = QR \ $gf[:y_hat]
    end samples=10 evals=1

    ls = LeastSquaresF(gf)

    t_lbfgs = @benchmark begin
        LimitedMemoryBFGS($ls)
    end samples=10 evals=1
    
    push!(accData[:m], m)
    push!(accData[:meantimeLBFGS], mean(t_lbfgs.times))
    push!(accData[:stdtimeLBFGS], std(t_lbfgs.times))
    push!(accData[:meanallocsLBFGS], mean(t_lbfgs.memory))
    push!(accData[:meantimeQR], mean(t_qr.times))
    push!(accData[:stdtimeQR], std(t_qr.times))
    push!(accData[:meanallocsQR], mean(t_qr.memory))
    println("Done: m " * string(m))
    flush(stdout)
end

CSV.write(outputvsc, DataFrame(accData));

In [ ]:
# QR vs LBFGS running time comparison with respect to m, fixing n, on ill-conditioned matrix
λ    = 1e-12
n    = 50
runs = 10

# output csv
outputvsc = joinpath(baseDir, "QRvsLBFGS-m-n" * string(n) * "-illcond--time.csv");
accData = Dict(
    :m   => Array{Float64}(undef, 0),
    :meantimeLBFGS => Array{Float64}(undef, 0),
    :stdtimeLBFGS => Array{Float64}(undef, 0),
    :meanallocsLBFGS => Array{Float64}(undef, 0),
    :meantimeQR => Array{Float64}(undef, 0),
    :stdtimeQR => Array{Float64}(undef, 0),
    :meanallocsQR => Array{Float64}(undef, 0)
    )

# warm up
qrfact([0. 1; 1 0]) \ [1.; 1]

for i ∈ 0:runs
    m = 500 + i * 500
    gf = genFunc(:exactRandDataset, λ=λ, m=m, n=n)

    t_qr = @benchmark begin
        QR = qrfact($gf[:X_hat])
        w = QR \ $gf[:y_hat]
    end samples=10 evals=1

    ls = LeastSquaresF(gf)

    t_lbfgs = @benchmark begin
        LimitedMemoryBFGS($ls)
    end samples=10 evals=1
    
    push!(accData[:m], m)
    push!(accData[:meantimeLBFGS], mean(t_lbfgs.times))
    push!(accData[:stdtimeLBFGS], std(t_lbfgs.times))
    push!(accData[:meanallocsLBFGS], mean(t_lbfgs.memory))
    push!(accData[:meantimeQR], mean(t_qr.times))
    push!(accData[:stdtimeQR], std(t_qr.times))
    push!(accData[:meanallocsQR], mean(t_qr.memory))
    println("Done: m " * string(m))
    flush(stdout)
end

CSV.write(outputvsc, DataFrame(accData));

## Comparison of all quasi newton methods

In [3]:
# Quasi newton methods running time comparison, fixing n and varying m, on ill-conditioned matrix
λ    = 1e-12
n    = 50
runs = 10
samples = 10

# output csv
outputvsc = joinpath(baseDir, "Quasi-Newton-Comparison-time-illcond.csv");
accData = Dict(
    :m   => Array{Float64}(undef, 0),

    :meantimeLBFGS => Array{Float64}(undef, 0),
    :stdtimeLBFGS => Array{Float64}(undef, 0),
    :accuracyLBFGS => Array{Float64}(undef, 0),
    :meanallocsLBFGS => Array{Float64}(undef, 0),

    :meantimeBFGS => Array{Float64}(undef, 0),
    :stdtimeBFGS => Array{Float64}(undef, 0),
    :accuracyBFGS => Array{Float64}(undef, 0),
    :meanallocsBFGS => Array{Float64}(undef, 0),

    :meantimeBFGSDogleg => Array{Float64}(undef, 0),
    :stdtimeBFGSDogleg => Array{Float64}(undef, 0),
    :accuracyBFGSDogleg => Array{Float64}(undef, 0),
    :meanallocsBFGSDogleg => Array{Float64}(undef, 0),

    :meantimeDFP => Array{Float64}(undef, 0),
    :stdtimeDFP => Array{Float64}(undef, 0),
    :accuracyDFP => Array{Float64}(undef, 0),
    :meanallocsDFP => Array{Float64}(undef, 0),

    :meantimeDFPDogleg => Array{Float64}(undef, 0),
    :stdtimeDFPDogleg => Array{Float64}(undef, 0),
    :accuracyDFPDogleg => Array{Float64}(undef, 0),
    :meanallocsDFPDogleg => Array{Float64}(undef, 0),

    :meantimeSR1 => Array{Float64}(undef, 0),
    :stdtimeSR1 => Array{Float64}(undef, 0),
    :accuracySR1 => Array{Float64}(undef, 0),
    :meanallocsSR1 => Array{Float64}(undef, 0)
    
    )


for i ∈ 0:runs
    m = 500 + i * 500

    gf = genFunc(:randDataset, λ=λ, m=m, n=n)
    ls = LeastSquaresF(gf)

    t_lbfgs = @benchmark begin
        LimitedMemoryBFGS($ls)
    end samples=samples evals=1
    res_lbfgs = LimitedMemoryBFGS(ls)


    BFGS.BFGSorDFP = :BFGS

    t_bfgs = @benchmark begin
        BroydenFletcherGoldfarbShanno($ls)
    end samples=samples evals=1
    res_bfgs = BroydenFletcherGoldfarbShanno(ls)

    t_bfgs_dogleg = @benchmark begin
        BroydenFletcherGoldfarbShannoDogleg($ls)
    end samples=samples evals=1
    res_bfgs_dogleg = BroydenFletcherGoldfarbShannoDogleg(ls)

    BFGS.BFGSorDFP = :DFP

    t_dfp = @benchmark begin
        BroydenFletcherGoldfarbShanno($ls)
    end samples=samples evals=1
    res_dfp = BroydenFletcherGoldfarbShanno(ls)

    t_dfp_dogleg = @benchmark begin
        BroydenFletcherGoldfarbShannoDogleg($ls)
    end samples=samples evals=1
    res_dfp_dogleg = BroydenFletcherGoldfarbShannoDogleg(ls)

    t_sr1 = @benchmark begin
        SymmetricRank1($ls)
    end samples=samples evals=1
    res_sr1 = SymmetricRank1(ls)
    
    push!(accData[:m], m)
    push!(accData[:meantimeLBFGS], mean(t_lbfgs.times))
    push!(accData[:stdtimeLBFGS], std(t_lbfgs.times))
    push!(accData[:accuracyLBFGS], norm(res_lbfgs.x - gf[:w_star]) / norm(gf[:w_star]))
    push!(accData[:meanallocsLBFGS], mean(t_lbfgs.memory))

    push!(accData[:meantimeBFGS], mean(t_bfgs.times))
    push!(accData[:stdtimeBFGS], std(t_bfgs.times))
    push!(accData[:accuracyBFGS], norm(res_bfgs.x - gf[:w_star]) / norm(gf[:w_star]))
    push!(accData[:meanallocsBFGS], mean(t_bfgs.memory))

    push!(accData[:meantimeBFGSDogleg], mean(t_bfgs_dogleg.times))
    push!(accData[:stdtimeBFGSDogleg], std(t_bfgs_dogleg.times))
    push!(accData[:accuracyBFGSDogleg], norm(res_bfgs_dogleg.x - gf[:w_star]) / norm(gf[:w_star]))
    push!(accData[:meanallocsBFGSDogleg], mean(t_bfgs_dogleg.memory))

    push!(accData[:meantimeDFP], mean(t_dfp.times))
    push!(accData[:stdtimeDFP], std(t_dfp.times))
    push!(accData[:accuracyDFP], norm(res_dfp.x - gf[:w_star]) / norm(gf[:w_star]))
    push!(accData[:meanallocsDFP], mean(t_dfp.memory))

    push!(accData[:meantimeDFPDogleg], mean(t_dfp_dogleg.times))
    push!(accData[:stdtimeDFPDogleg], std(t_dfp_dogleg.times))
    push!(accData[:accuracyDFPDogleg], norm(res_dfp_dogleg.x - gf[:w_star]) / norm(gf[:w_star]))
    push!(accData[:meanallocsDFPDogleg], mean(t_dfp_dogleg.memory))

    push!(accData[:meantimeSR1], mean(t_sr1.times))
    push!(accData[:stdtimeSR1], std(t_sr1.times))
    push!(accData[:accuracySR1], norm(res_sr1.x - gf[:w_star]) / norm(gf[:w_star]))
    push!(accData[:meanallocsSR1], mean(t_sr1.memory))
    println("Done: m " * string(m))
    flush(stdout)
end

CSV.write(outputvsc, DataFrame(accData));

Done: m 500
Done: m 1000
Done: m 1500
Done: m 2000
Done: m 2500
Done: m 3000
Done: m 3500
Done: m 4000
Done: m 4500
Done: m 5000
Done: m 5500


In [2]:
# Quasi newton methods running time comparison, fixing n and varying m, on well-conditioned matrix
λ    = 1e-4
n    = 50
runs = 10
samples = 10

# output csv
outputvsc = joinpath(baseDir, "Quasi-Newton-Comparison-time-wellcond.csv");
accData = Dict(
    :m   => Array{Float64}(undef, 0),

    :meantimeLBFGS => Array{Float64}(undef, 0),
    :stdtimeLBFGS => Array{Float64}(undef, 0),
    :accuracyLBFGS => Array{Float64}(undef, 0),
    :meanallocsLBFGS => Array{Float64}(undef, 0),

    :meantimeBFGS => Array{Float64}(undef, 0),
    :stdtimeBFGS => Array{Float64}(undef, 0),
    :accuracyBFGS => Array{Float64}(undef, 0),
    :meanallocsBFGS => Array{Float64}(undef, 0),

    :meantimeBFGSDogleg => Array{Float64}(undef, 0),
    :stdtimeBFGSDogleg => Array{Float64}(undef, 0),
    :accuracyBFGSDogleg => Array{Float64}(undef, 0),
    :meanallocsBFGSDogleg => Array{Float64}(undef, 0),

    :meantimeDFP => Array{Float64}(undef, 0),
    :stdtimeDFP => Array{Float64}(undef, 0),
    :accuracyDFP => Array{Float64}(undef, 0),
    :meanallocsDFP => Array{Float64}(undef, 0),

    :meantimeDFPDogleg => Array{Float64}(undef, 0),
    :stdtimeDFPDogleg => Array{Float64}(undef, 0),
    :accuracyDFPDogleg => Array{Float64}(undef, 0),
    :meanallocsDFPDogleg => Array{Float64}(undef, 0),

    :meantimeSR1 => Array{Float64}(undef, 0),
    :stdtimeSR1 => Array{Float64}(undef, 0),
    :accuracySR1 => Array{Float64}(undef, 0),
    :meanallocsSR1 => Array{Float64}(undef, 0)
    
    )


for i ∈ 0:runs
    m = 500 + i * 500

    gf = genFunc(:randDataset, λ=λ, m=m, n=n)
    ls = LeastSquaresF(gf)

    t_lbfgs = @benchmark begin
        LimitedMemoryBFGS($ls)
    end samples=samples evals=1
    res_lbfgs = LimitedMemoryBFGS(ls)


    BFGS.BFGSorDFP = :BFGS

    t_bfgs = @benchmark begin
        BroydenFletcherGoldfarbShanno($ls)
    end samples=samples evals=1
    res_bfgs = BroydenFletcherGoldfarbShanno(ls)

    t_bfgs_dogleg = @benchmark begin
        BroydenFletcherGoldfarbShannoDogleg($ls)
    end samples=samples evals=1
    res_bfgs_dogleg = BroydenFletcherGoldfarbShannoDogleg(ls)

    BFGS.BFGSorDFP = :DFP

    t_dfp = @benchmark begin
        BroydenFletcherGoldfarbShanno($ls)
    end samples=samples evals=1
    res_dfp = BroydenFletcherGoldfarbShanno(ls)

    t_dfp_dogleg = @benchmark begin
        BroydenFletcherGoldfarbShannoDogleg($ls)
    end samples=samples evals=1
    res_dfp_dogleg = BroydenFletcherGoldfarbShannoDogleg(ls)

    t_sr1 = @benchmark begin
        SymmetricRank1($ls)
    end samples=samples evals=1
    res_sr1 = SymmetricRank1(ls)
    
    push!(accData[:m], m)
    push!(accData[:meantimeLBFGS], mean(t_lbfgs.times))
    push!(accData[:stdtimeLBFGS], std(t_lbfgs.times))
    push!(accData[:accuracyLBFGS], norm(res_lbfgs.x - gf[:w_star]) / norm(gf[:w_star]))
    push!(accData[:meanallocsLBFGS], mean(t_lbfgs.memory))

    push!(accData[:meantimeBFGS], mean(t_bfgs.times))
    push!(accData[:stdtimeBFGS], std(t_bfgs.times))
    push!(accData[:accuracyBFGS], norm(res_bfgs.x - gf[:w_star]) / norm(gf[:w_star]))
    push!(accData[:meanallocsBFGS], mean(t_bfgs.memory))

    push!(accData[:meantimeBFGSDogleg], mean(t_bfgs_dogleg.times))
    push!(accData[:stdtimeBFGSDogleg], std(t_bfgs_dogleg.times))
    push!(accData[:accuracyBFGSDogleg], norm(res_bfgs_dogleg.x - gf[:w_star]) / norm(gf[:w_star]))
    push!(accData[:meanallocsBFGSDogleg], mean(t_bfgs_dogleg.memory))

    push!(accData[:meantimeDFP], mean(t_dfp.times))
    push!(accData[:stdtimeDFP], std(t_dfp.times))
    push!(accData[:accuracyDFP], norm(res_dfp.x - gf[:w_star]) / norm(gf[:w_star]))
    push!(accData[:meanallocsDFP], mean(t_dfp.memory))

    push!(accData[:meantimeDFPDogleg], mean(t_dfp_dogleg.times))
    push!(accData[:stdtimeDFPDogleg], std(t_dfp_dogleg.times))
    push!(accData[:accuracyDFPDogleg], norm(res_dfp_dogleg.x - gf[:w_star]) / norm(gf[:w_star]))
    push!(accData[:meanallocsDFPDogleg], mean(t_dfp_dogleg.memory))

    push!(accData[:meantimeSR1], mean(t_sr1.times))
    push!(accData[:stdtimeSR1], std(t_sr1.times))
    push!(accData[:accuracySR1], norm(res_sr1.x - gf[:w_star]) / norm(gf[:w_star]))
    push!(accData[:meanallocsSR1], mean(t_sr1.memory))
    println("Done: m " * string(m))
    flush(stdout)
end

CSV.write(outputvsc, DataFrame(accData));

Done: m 500
Done: m 1000
Done: m 1500
Done: m 2000
Done: m 2500
Done: m 3000
Done: m 3500
Done: m 4000
Done: m 4500
Done: m 5000
Done: m 5500
